<a href="https://colab.research.google.com/github/Alanapengilley/Dampier/blob/main/Dampier_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#load in packages required 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Geological dataset consists of 314 geological samples from northern New Britain, with geochemical and social region classes attributed. 

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Source_data.csv to Source_data.csv


In [3]:
#load in geological dataset 
import io
source_data = pd.read_csv(io.BytesIO(uploaded['Source_data.csv']))

16 elements were employed in analysis, MgO and Nb were excluded due to missing values. 

In [ ]:
#X = source_data.iloc[:,5:21].values
#y = source_data.iloc[:,3].values

In [4]:
X = source_data[['Al2O3', 'SiO2', 'P2O5', 'K2O', 'CaO', 'TiO2', 'V', 'MnO', 'FeO2O3', 'Ni', 'Cu', 'Zn', 'Rb', 'Sr', 'Y', 'Zr']]
y = source_data['Geochem'] #divide into geochemical and social classes 
Z = source_data['Social']

In [ ]:
print(X)

In [6]:
y.unique()

array(['E', 'F', 'GN', 'GS', 'H', 'Rabaul', 'Vitiaz', 'New Guinea'],
      dtype=object)

In [7]:
Z.unique()

array(['Hoskins', 'Eastern', 'W.P.S', 'W.P.N', 'Bali-Witu', 'Rabaul',
       'Vitiaz', 'New Guinea'], dtype=object)

Artefact data contains 17 elemenets collected from analysis of dampier axe. Nb was excluded to match geoloical data set. 

In [8]:
from google.colab import files
uploaded = files.upload()

Saving dampier_data.csv to dampier_data.csv


In [9]:
#load in artefact data 
dampier_data = pd.read_csv(io.BytesIO(uploaded['dampier_data.csv']))

In [ ]:
#testX = dampier_data.iloc[:,3:19].values
#print(testX)

In [10]:
dampier_axe = dampier_data[['Al2O3', 'SiO2', 'P2O5', 'K2O', 'CaO', 'TiO2', 'V', 'MnO', 'FeO2O3', 'Ni', 'Cu', 'Zn', 'Rb', 'Sr', 'Y', 'Zr']]
testX = dampier_axe.values.tolist()
print(testX)

[[15.3, 36.5, 0.0, 0.5, 10.9, 0.6, 98.8, 0.1, 9.8, 63.0, 72.0, 87.0, 8.0, 276.0, 19.0, 13.0]]


In [ ]:
dampier_axe.info()

In [12]:
from sklearn.preprocessing import StandardScaler #data scaled 
sc = StandardScaler()
scaler = sc.fit(X)
trainX_scaled = scaler.transform(X)
testX_scaled = scaler.transform(testX)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [13]:
#first model for grouping of geochemical classes 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 3)
model = lda.fit(trainX_scaled, y)
print(model)

LinearDiscriminantAnalysis(n_components=3)


In [14]:
#second model for grouping of social classes 
lda_2 = LDA(n_components = 3)
model_2 = lda_2.fit(trainX_scaled, Z)
print(model_2)

LinearDiscriminantAnalysis(n_components=3)


In [15]:
lda.explained_variance_ratio_.sum()

0.8801229176051104

In [16]:
lda_2.explained_variance_ratio_.sum()

0.8936563494194941

In [17]:
#training and test data tranformed 
trainX_lda = lda.transform(trainX_scaled) #geochemical
testX_lda =lda.transform(testX_scaled)

In [ ]:
print(trainX_lda)

In [19]:
print(testX_lda)

[[-3.06458021  1.04757773  1.94687777]]


In [69]:
trainX_lda_2 = lda_2.transform(trainX_scaled)
testX_lda_2 = lda_2.transform(testX_scaled)

In [20]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

#Define method to evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

#evaluate model
scores = cross_val_score(model, trainX_scaled, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(np.mean(scores))   
print(scores) #accuracy scores 

0.8599462365591397
[0.9375     0.71875    0.875      0.8125     0.80645161 0.90322581
 0.90322581 0.90322581 0.87096774 0.90322581 0.90625    0.875
 0.9375     0.90625    0.77419355 0.77419355 0.74193548 0.87096774
 0.90322581 0.87096774 0.875      0.71875    0.75       0.9375
 0.90322581 0.87096774 0.90322581 0.87096774 0.83870968 0.93548387]


In [21]:
scores_2 = cross_val_score(model, trainX_scaled, Z, scoring='accuracy', cv=cv, n_jobs=-1)
print(np.mean(scores_2))   
print(scores_2) #accuracy scores 

0.8194220430107527
[0.78125    0.875      0.90625    0.65625    0.93548387 0.77419355
 0.83870968 0.70967742 0.83870968 0.83870968 0.84375    0.8125
 0.8125     0.90625    0.87096774 0.74193548 0.93548387 0.80645161
 0.77419355 0.70967742 0.75       0.90625    0.8125     0.875
 0.77419355 0.74193548 0.87096774 0.90322581 0.74193548 0.83870968]


In [ ]:
#from sklearn.linear_model import LogisticRegression
#logReg = LogisticRegression()
#logReg.fit(trainX_lda, y)

## Geochemical Region Prediction 

Predict() can be executed on a trained model in order to predict the actual label over a new set of data. 

In [22]:
from sklearn.metrics import accuracy_score
y_pred_label = model.predict(testX_scaled)
print(y_pred_label)

['E']


Predict_proba() returns the class probabilities for each data point. Classes and probability scores are listed in the same order

In [23]:
y_pred_test = model.predict_proba(testX_scaled)
classes = model.classes_
print(y_pred_test)
print(classes)

#for class_name, proba in zip(classes, y_pred_test):
    #print(f"{class_name}: {proba}")


[[9.51162627e-01 9.32921155e-03 3.94713446e-02 3.68168663e-05
  2.29485346e-10 2.62946540e-48 1.86576651e-10 5.20298859e-24]]
['E' 'F' 'GN' 'GS' 'H' 'New Guinea' 'Rabaul' 'Vitiaz']


In [24]:
np.max(model.predict_proba(testX_scaled)) #E

0.9511626265533794

## Social Region Prediction 

In [25]:
Z_pred_label = model_2.predict(testX_scaled)
print(Z_pred_label)

['Hoskins']


In [26]:
Z_pred_test = model_2.predict_proba(testX_scaled)
classes = model_2.classes_
print(Z_pred_test)
print(classes)

[[2.63156001e-10 1.68762466e-01 5.32921839e-01 2.46019882e-44
  5.72102884e-10 1.79677674e-22 2.66706379e-01 3.16093162e-02]]
['Bali-Witu' 'Eastern' 'Hoskins' 'New Guinea' 'Rabaul' 'Vitiaz' 'W.P.N'
 'W.P.S']


In [27]:
np.max(model_2.predict_proba(testX_scaled)) #Hoskins

0.5329218386092035

Prediction of Geochemical class with training data

In [28]:
#y_pred_train = logReg.predict(trainX_lda)
y_pred = model.predict(trainX_scaled)
y_pred_train = model.predict_proba(trainX_scaled)

print('Accuracy: {:.2f}'.format(accuracy_score(y, y_pred)))

Accuracy: 0.90


Preidction of Social class with training data 

In [29]:
Z_pred = model_2.predict(trainX_scaled)
Z_pred_train = model_2.predict_proba(trainX_scaled)

print('Accuracy: {:.2f}'.format(accuracy_score(Z, Z_pred)))

Accuracy: 0.86


## Data visualisation 

In [41]:
y_pred_train

(314, 8)

In [64]:
print(model.get_params()) 

highest_prob = [] #function for highest probability of sample to geochemical class 

for sample in range(y_pred_train.shape[0]):    #iterates through each sample
    highest_prob.append(np.max(y_pred_train[sample]))   #gets max probability of that sample (highest likelyhood)
    
    
print(len(highest_prob))

{'covariance_estimator': None, 'n_components': 3, 'priors': None, 'shrinkage': None, 'solver': 'svd', 'store_covariance': False, 'tol': 0.0001}
314


In [ ]:
Z_pred_train

In [63]:
highest_prob_2 = []

for sample in range(Z_pred_train.shape[0]):    #iterates through each sample
    highest_prob_2.append(np.max(Z_pred_train[sample]))   #gets max probability of that sample (highest likelyhood)
    
    
print(len(highest_prob_2))

314


Geochem regions 

In [ ]:
df = pd.DataFrame({'sample':source_data.Name.values, 'predicted_cat_geochem':y_pred, 'prob_predicted':highest_prob, 'actual_cat_geochem':source_data.Geochem, 'predicted_cat_social': Z_pred, 'prob_predicted_2':highest_prob_2, 'actual_cat_social': source_data.Social})
                  

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    print(df)

In [66]:
print(df.columns.tolist())

['sample', 'predicted_cat_geochem', 'prob_predicted', 'actual_cat_geochem', 'predicted_cat_social', 'prob_predicted_2', 'actual_cat_social']


In [73]:
#df.to_csv (r'C:\Users\AlanaPengilley\Desktop\df.csv', index = False, header=True)
df.to_csv("results.csv")

In [56]:
df_2 = pd.DataFrame({'sample':dampier_data.Name.values, 'predicted_geochem':y_pred_label, 'predicted_social': Z_pred_label})
print(df_2)

        sample predicted_geochem predicted_social
0  Dampier_Axe                 E          Hoskins


Geochemical classes 

In [57]:
print('classes: ', lda.classes_)
print('Priors: ', lda.priors_)
print('Explained variance ratio: ', lda.explained_variance_ratio_)

classes:  ['E' 'F' 'GN' 'GS' 'H' 'New Guinea' 'Rabaul' 'Vitiaz']
Priors:  [0.27388535 0.20063694 0.10509554 0.06687898 0.08280255 0.05095541
 0.17197452 0.0477707 ]
Explained variance ratio:  [0.49708357 0.23986742 0.14317192]


Social Classes

In [58]:
print('classes: ', lda_2.classes_)
print('Priors: ', lda_2.priors_)
print('Explained variance ratio: ', lda_2.explained_variance_ratio_)

classes:  ['Bali-Witu' 'Eastern' 'Hoskins' 'New Guinea' 'Rabaul' 'Vitiaz' 'W.P.N'
 'W.P.S']
Priors:  [0.08280255 0.24840764 0.20382166 0.05095541 0.17197452 0.0477707
 0.10509554 0.08917197]
Explained variance ratio:  [0.49315306 0.24871348 0.15178982]


## Plots

In [ ]:
import plotly.express as px
fig = px.scatter(source_data, x=trainX_lda[:,0], y=trainX_lda[:,1],
                 labels = {
                     "x": "LDA 1",
                     "y": 'LDA 2',
                 },
                 opacity=1, color=y)

# Set figure title
fig.update_layout(title_text="LDA scatter plot of training data")

# Update marker size
fig.update_traces(marker=dict(size=4))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.show()

In [71]:
import plotly.express as px
fig = px.scatter(source_data, x=trainX_lda_2[:,0], y=trainX_lda_2[:,1],
                 labels = {
                     "x": "LDA 1",
                     "y": 'LDA 2',
                 },
                 opacity=1, color=Z)

# Set figure title
fig.update_layout(title_text="Social class model - LDA scatter plot of training data")

# Update marker size
fig.update_traces(marker=dict(size=4))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.show()